# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [16]:
# Your code here
import sqlite3
from sql_utils import fetch2frame

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()
cur.execute("""SELECT name AS table_name FROM sqlite_master;""")
fetch2frame(cur)

,table_name
0,productlines
1,offices
2,customers
3,employees
4,orderdetails
5,orders
6,payments
7,products


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [20]:
# Your code here
cur.execute("""SELECT lastName, firstName FROM employees JOIN offices 
               ON employees.officeCode = offices.officeCode
               WHERE offices.city = 'Boston';""")
fetch2frame(cur)

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [33]:
# Your code here
cur.execute("""SELECT COUNT(DISTINCT employeeNumber) AS employeeCount,
               employees.officeCode
               FROM employees JOIN offices 
               ON employees.officeCode = offices.officeCode
               GROUP BY employees.officeCode
               HAVING employeeCount = 0;""")
fetch2frame(cur)

,employeeCount,officeCode


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [51]:
cur.execute("""SELECT * FROM customers;""")
customer_df = fetch2frame(cur)
customer_df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700


In [52]:
cur.execute("""SELECT * FROM offices;""")
office_df = fetch2frame(cur)
office_df.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan


In [67]:
# Display the total number of customers in each city with an office
cur.execute("""SELECT COUNT(DISTINCT customerNumber) as totalCustomers,
               offices.city
               FROM customers INNER JOIN offices ON customers.city == offices.city
               GROUP BY offices.city
               ORDER BY totalCustomers DESC;""")
fetch2frame(cur)

,totalCustomers,city
0,5,NYC
1,3,Paris
2,2,San Francisco
3,2,London
4,2,Boston


In [78]:
# Display each customer's name and their total number of orders
cur.execute("""SELECT customers.customerName,
               COUNT(DISTINCT orders.orderNumber) AS orderCount
               FROM customers JOIN orders
               ON customers.customerNumber = orders.customerNumber
               GROUP BY customers.customerNumber
               ORDER BY orderCount DESC;""")
fetch2frame(cur)

,customerName,orderCount
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,Reims Collectables,5
3,"Down Under Souveniers, Inc",5
4,"Dragon Souveniers, Ltd.",5
...,...,...
93,"Clover Collections, Co.",2
94,Osaka Souveniers Co.,2
95,Cambridge Collectables Co.,2
96,Daedalus Designs Imports,2


In [89]:
# Display each customer's name, their total number of orders, and their sales rep's last name
cur.execute("""SELECT customers.customerName,
               employees.LastName,
               COUNT(DISTINCT orders.orderNumber) AS orderCount
               FROM customers JOIN orders
               ON customers.customerNumber = orders.customerNumber
               JOIN employees ON customers.salesRepEmployeeNumber = employees.employeeNumber
               GROUP BY customers.customerNumber
               ORDER BY orderCount DESC;""")
fetch2frame(cur)

,customerName,lastName,orderCount
0,Euro+ Shopping Channel,Hernandez,26
1,Mini Gifts Distributors Ltd.,Jennings,17
2,Reims Collectables,Bondur,5
3,"Down Under Souveniers, Inc",Marsh,5
4,"Dragon Souveniers, Ltd.",Nishi,5
...,...,...,...
93,"Clover Collections, Co.",Jones,2
94,Osaka Souveniers Co.,Nishi,2
95,Cambridge Collectables Co.,Firrelli,2
96,Daedalus Designs Imports,Hernandez,2


## Level Up: Display the names of every individual product that each employee has sold

In [107]:
# Your code here
cur.execute("""SELECT employees.employeeNumber, employees.lastName AS employeeName, productName
               FROM products
               JOIN orderDetails ON products.productCode = orderDetails.productCode
               JOIN orders ON orderDetails.orderNumber = orders.orderNumber
               JOIN customers ON orders.customerNumber = customers.customerNumber
               JOIN employees ON customers.salesRepEmployeeNumber = employees.employeeNumber
               ORDER BY employees.employeeNumber;""")
fetch2frame(cur)

,employeeNumber,employeeName,productName
0,1165,Jennings,1969 Harley Davidson Ultimate Chopper
1,1165,Jennings,1969 Harley Davidson Ultimate Chopper
2,1165,Jennings,1969 Harley Davidson Ultimate Chopper
3,1165,Jennings,1952 Alpine Renault 1300
4,1165,Jennings,1952 Alpine Renault 1300
...,...,...,...
2991,1702,Gerard,The Queen Mary
2992,1702,Gerard,The Queen Mary
2993,1702,Gerard,Boeing X-32A JSF
2994,1702,Gerard,Pont Yacht


## Level Up: Display the Number of Products each employee has sold

In [112]:
# Your code here
cur.execute("""SELECT employees.employeeNumber, employees.lastName AS employeeName, COUNT(employees.employeeNumber) as totalSales
               FROM products
               JOIN orderDetails ON products.productCode = orderDetails.productCode
               JOIN orders ON orderDetails.orderNumber = orders.orderNumber
               JOIN customers ON orders.customerNumber = customers.customerNumber
               JOIN employees ON customers.salesRepEmployeeNumber = employees.employeeNumber
               GROUP BY employees.employeeNumber
               ORDER BY totalSales DESC;""")
fetch2frame(cur)

,employeeNumber,employeeName,totalSales
0,1370,Hernandez,396
1,1165,Jennings,331
2,1401,Castillo,272
3,1501,Bott,236
4,1504,Jones,220
5,1323,Vanauf,211
6,1612,Marsh,185
7,1611,Fixter,185
8,1337,Bondur,177
9,1216,Patterson,152


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!